In [206]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph 
from datetime import date
import random

from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader

%matplotlib inline

import random 
random.seed(42)

In [126]:
csv_path_1 = "training_dataset.csv"
csv_path_2 = "score.csv"

# 0. First, let's understand our data...

In [127]:
df = pd.read_csv(csv_path_1) # load the pandas dataframe
df_score = pd.read_csv(csv_path_2)
initial_cols_to_drop = ["Unnamed: 0","Unnamed: 0.1", "period", "test", "recent_date", "date"] 
for col_name in initial_cols_to_drop: # drops columns that aren't supposed to be in dataset
    try:
        df = df.drop(columns=[col_name])
        df_score = df_score.drop(columns=[col_name])
    except:
        continue
#df = df.rename(columns={"Unnamed: 0.1": "TODO_FIND_COLUMN_NAME_2"})
display(df.head())

,dt,weekday,year,id_driver,id_carrier_number,dim_carrier_type,dim_carrier_company_name,home_base_city,home_base_state,carrier_trucks,...,most_recent_load_date,load_day,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads
0,2019-12-16,Monday,2019,21350,U0109015,Owner Operator,CA&F TRUCKING,Maywood,CA,"[""poweronly""]",...,2021-02-17,2019-12-16,2,0,438,438,0,45,45,483
1,2021-01-15,Friday,2021,36437,C0097727,Fleet,New opportunities inc,Los Angeles,CA,"[""poweronly"", ""boxtruck""]",...,2021-02-03,2021-01-15,1,2,72,74,0,1,1,75
2,2019-12-26,Thursday,2019,19323,U0107081,Owner Operator,RAS,Compton,CA,"[""poweronly""]",...,2020-09-25,2019-12-26,1,0,180,180,0,2,2,182
3,2021-02-10,Wednesday,2021,34809,C0094651,Fleet,NFS asset Drayage,Lynwood,CA,"[""poweronly"", ""dryvan""]",...,2021-02-17,2021-02-10,3,0,0,0,0,0,0,62
4,2017-07-24,Monday,2017,4728,U0094376,Owner Operator,joes transportation,Norco,CA,"[""dryvan""]",...,2017-10-11,2017-07-24,2,57,0,57,314,0,314,371


In [128]:
df.describe()

,year,id_driver,num_trucks,days_signup_to_approval,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads
count,83414.000000,83414.000000,83344.000000,71124.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000
mean,2018.960930,18222.414954,22.582921,298.752489,2.076270,29.477762,71.579675,101.057436,148.258422,13.073021,161.331443,266.502661
std,1.359343,11667.704926,48.829719,390.345107,2.672163,88.171940,194.532776,214.502147,415.978060,42.241592,413.792137,448.806175
min,2015.000000,20.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2018.000000,7890.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,37.000000
50%,2019.000000,16299.000000,4.000000,61.000000,1.000000,2.000000,0.000000,13.000000,15.000000,0.000000,37.000000,110.000000
75%,2020.000000,28974.000000,14.000000,497.000000,2.000000,23.000000,18.000000,94.000000,112.000000,1.000000,135.000000,325.000000
max,2021.000000,38125.000000,195.000000,1653.000000,129.000000,902.000000,1324.000000,1348.000000,4266.000000,371.000000,4266.000000,4266.000000


In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83414 entries, 0 to 83413
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   dt                        83414 non-null  object 
 1   weekday                   83414 non-null  object 
 2   year                      83414 non-null  int64  
 3   id_driver                 83414 non-null  int64  
 4   id_carrier_number         83414 non-null  object 
 5   dim_carrier_type          83414 non-null  object 
 6   dim_carrier_company_name  83365 non-null  object 
 7   home_base_city            83369 non-null  object 
 8   home_base_state           83369 non-null  object 
 9   carrier_trucks            83414 non-null  object 
 10  num_trucks                83344 non-null  float64
 11  interested_in_drayage     83414 non-null  object 
 12  port_qualified            83414 non-null  object 
 13  signup_source             83414 non-null  object 
 14  ts_sig

# 1. Generate Labels

In [130]:
# converts date from csv to a python datetime object making it easier to work with
def convert_dates(df):
    dates_columns = ['most_recent_load_date', 'first_load_date', 'load_day', 'dt']
    for col_name in dates_columns:
        try:
            df[col_name] = pd.to_datetime(df[col_name], format='%Y-%m-%d')
        except:
            continue

convert_dates(df)
convert_dates(df_score)

In [131]:
df['most_recent_load_date'].head()

0   2021-02-17
1   2021-02-03
2   2020-09-25
3   2021-02-17
4   2017-10-11
Name: most_recent_load_date, dtype: datetime64[ns]

In [132]:
total_loads75 = df.total_loads.quantile(0.75) # finds 75th percentile of loads
most_recent_load_date75 = df.most_recent_load_date.quantile(0.75) # finds 75th percentile of most recent load date

print(total_loads75)
print(most_recent_load_date75)


# Manual Check
# sorted_dts = sorted(list(df.most_recent_load_date))
# quartile_estimate_index = int(len(sorted_dts)*0.75)
# print("SORTED INDEX", sorted_dts[quartile_estimate_index])

325.0
2021-02-14 00:00:00


In [133]:
# new_labels = {"label": {}}
num_days_worked_dict = {}

for index, row in df.iterrows(): # changes the labels in the label columns
    # checks if the load and most recent load date are in the 75th percentile
    if row["total_loads"] >= total_loads75 and row["most_recent_load_date"] >= most_recent_load_date75:
        df.at[index, "label"] = 1
    else:
        df.at[index, "label"] = 0

In [134]:
uniqueValues, occurCount = np.unique(np.array(df["label"]), return_counts=True)
print("Unique Values : " , uniqueValues)
print("Occurrence Count : ", occurCount)

Unique Values :  [0. 1.]
Occurrence Count :  [73021 10393]


In [135]:
loc_arr = []
for index, row in df.iterrows():
    if pd.isnull(row["home_base_city"]) != True and pd.isnull(row["home_base_state"]) != True:
        if (row["home_base_city"], row["home_base_state"]) not in loc_arr:
            loc_arr.append((row["home_base_city"], row["home_base_state"]))
            
# for index, row in df.iterrows():
#     try:
#         x = len(row["home_base_city"])
#         x = len(row["home_base_state"])
#     except:
#         index_num = random.randint(0, len(loc_arr)-1)
#         df.at[index, "home_base_city"] = loc_arr[index_num][0]
#         df.at[index, "home_base_state"] = loc_arr[index_num][1]

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83414 entries, 0 to 83413
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   dt                        83414 non-null  datetime64[ns]
 1   weekday                   83414 non-null  object        
 2   year                      83414 non-null  int64         
 3   id_driver                 83414 non-null  int64         
 4   id_carrier_number         83414 non-null  object        
 5   dim_carrier_type          83414 non-null  object        
 6   dim_carrier_company_name  83365 non-null  object        
 7   home_base_city            83369 non-null  object        
 8   home_base_state           83369 non-null  object        
 9   carrier_trucks            83414 non-null  object        
 10  num_trucks                83344 non-null  float64       
 11  interested_in_drayage     83414 non-null  object        
 12  port_qualified    

In [137]:
groups = df.groupby("id_driver")

In [138]:
# NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_arr = []
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    temp_arr = []
    temp_arr.append(key)
    
    if group["dim_carrier_type"].nunique() == 2:
        temp_arr.append("Both")
    elif group["dim_carrier_type"].nunique() == 0:
        temp_arr.append(None)
    else:
        temp_arr.append((group["dim_carrier_type"].iloc[0]))
    
    
    idxmax_cols = ["dim_carrier_company_name", 
                   "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                  "days_signup_to_approval"] #"home_base_city", "home_base_state",
    
    for col in idxmax_cols:
        try:
            temp_arr.append(group[col].value_counts().dropna(how="any").idxmax())
        except:
            
            temp_arr.append(None)
    
    try:
        x = len(group["home_base_city"].dropna(how="any").iloc[0])
        x = len(group["home_base_state"].dropna(how="any").iloc[0])
        temp_arr.append(group["home_base_city"].dropna(how="any").iloc[0])
        temp_arr.append(group["home_base_state"].dropna(how="any").iloc[0])
    except:
        index_num = random.randint(0, len(loc_arr)-1)
        temp_arr.append(loc_arr[index_num][0])
        temp_arr.append(loc_arr[index_num][1])
        
    
    
    try:
        temp_arr.append(group["num_trucks"].dropna(how="any").mean())
    except:
        temp_arr.append(None)
        
    iloc_cols = ["interested_in_drayage", "port_qualified", "driver_with_twic", 
                 "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                 "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                 "brokerage_loads_atlas", "brokerage_loads", "total_loads"]
    for col in iloc_cols:
        try:
            temp_arr.append(group[col].dropna(how="any").iloc[0])
        except:
            temp_arr.append(None)
    
    temp_arr.append(group.shape[0])
    
    temp_arr.append(((pd.to_datetime(date.today()) - group["most_recent_load_date"].dropna(how="any").iloc[0]).days) * (-1))
    
    temp_arr.append(group["label"].value_counts().dropna(how="any").idxmax())
    
    new_arr.append(np.array(temp_arr))

In [139]:
column_names = ["id_driver", "dim_carrier_type", "dim_carrier_company_name", 
                "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                "days_signup_to_approval", "home_base_city", "home_base_state", "num_trucks", 
                "interested_in_drayage", "port_qualified", "driver_with_twic", 
                "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                "brokerage_loads_atlas", "brokerage_loads", "total_loads", "num_trips_made", 
                "days_since_last_load", "label"]

df = pd.DataFrame(np.array(new_arr), columns=column_names)

In [140]:
# convert = ["id_driver", "days_signup_to_approval", "marketplace_loads_otr", 
#                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
#                "brokerage_loads_atlas", "brokerage_loads", "total_loads", "num_trips_made", "label",
#                "num_trucks", "dim_carrier_type", "dim_carrier_company_name", "home_base_city", "home_base_state",
#                "interested_in_drayage", "port_qualified", "signup_source", "driver_with_twic"]
for col in column_names:
    try:
        df[col] = df[col].convert_dtypes()
    except:
        continue

In [141]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_dict = {}
label_dict = {}
most_recent_date_arr = []
num_trips_arr = []
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    if key not in new_dict:
        try:
            new_dict[key] = (group["load_day"].dropna(how="any").iloc[0], group.shape[0])
        except:
            new_dict[key] = None
        
        try:
            label_dict[key] = group["label"].dropna(how="any").iloc[0]
        except:
            label_dict[key] = 1

# for index, row in df.iterrows():
#     most_recent_date_arr.append(new_dict[row["id_driver"]][0])
#     num_trips_arr.append(new_dict[row["id_driver"]][1])
# df["num_trips_made"] = np.array(num_trips_arr)

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 5291 non-null   Int64         
 1   dim_carrier_type          5291 non-null   string        
 2   dim_carrier_company_name  5284 non-null   string        
 3   carrier_trucks            5291 non-null   string        
 4   signup_source             5291 non-null   string        
 5   ts_signup                 5291 non-null   string        
 6   ts_first_approved         3962 non-null   string        
 7   days_signup_to_approval   3962 non-null   Int64         
 8   home_base_city            5291 non-null   string        
 9   home_base_state           5291 non-null   string        
 10  num_trucks                5249 non-null   float64       
 11  interested_in_drayage     5291 non-null   string        
 12  port_qualified      

In [143]:
df["label"].describe()

count    5291.000000
mean        0.010395
std         0.101434
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [144]:
groups = df_score.groupby("id_driver")

In [145]:
# NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_arr = []
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    temp_arr = []
    temp_arr.append(key)
    
    if group["dim_carrier_type"].nunique() == 2:
        temp_arr.append("Both")
    elif group["dim_carrier_type"].nunique() == 0:
        temp_arr.append(None)
    else:
        temp_arr.append((group["dim_carrier_type"].iloc[0]))
    
    
    idxmax_cols = ["dim_carrier_company_name", 
                   "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                  "days_signup_to_approval"]
    
    for col in idxmax_cols:
        try:
            temp_arr.append(group[col].value_counts().dropna(how="any").idxmax())
        except:
            temp_arr.append(None)
            
    
    try:
        x = len(group["home_base_city"].dropna(how="any").iloc[0])
        x = len(group["home_base_state"].dropna(how="any").iloc[0])
        temp_arr.append(group["home_base_city"].dropna(how="any").iloc[0])
        temp_arr.append(group["home_base_state"].dropna(how="any").iloc[0])
    except:
        index_num = random.randint(0, len(loc_arr)-1)
        temp_arr.append(loc_arr[index_num][0])
        temp_arr.append(loc_arr[index_num][1])
    
    
    try:
        temp_arr.append(group["num_trucks"].dropna(how="any").mean())
    except:
        temp_arr.append(None)
        
    iloc_cols = ["interested_in_drayage", "port_qualified", "driver_with_twic", 
                 "first_load_date", "load_day", "marketplace_loads_otr", 
                 "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                 "brokerage_loads_atlas", "brokerage_loads"]
    for col in iloc_cols:
        try:
            temp_arr.append(group[col].dropna(how="any").iloc[0])
        except:
            temp_arr.append(None)
            
    temp_arr.append(group.shape[0])
    
    temp_arr.append(((pd.to_datetime(date.today()) - group["load_day"].dropna(how="any").iloc[0]).days) * (-1))
    
    new_arr.append(np.array(temp_arr))

In [146]:
column_names = ["id_driver", "dim_carrier_type", "dim_carrier_company_name", 
                "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
                "days_signup_to_approval", "home_base_city", "home_base_state", "num_trucks", 
                "interested_in_drayage", "port_qualified", "driver_with_twic", 
                "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
                "brokerage_loads_atlas", "brokerage_loads", "num_trips_made", 
                "days_since_last_load"]

df_score = pd.DataFrame(np.array(new_arr), columns=column_names)

In [147]:
# convert = ["id_driver", "days_signup_to_approval", "marketplace_loads_otr", 
#                "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
#                "brokerage_loads_atlas", "brokerage_loads", "num_trips_made",
#                "num_trucks", "dim_carrier_type", "dim_carrier_company_name", "home_base_city", "home_base_state",
#                "interested_in_drayage", "port_qualified", "signup_source", "driver_with_twic"]
for col in column_names:
    try:
        df_score[col] = df_score[col].convert_dtypes()
    except:
        continue

In [148]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads
new_dict = {}
most_recent_date_arr = []
num_trips_arr = []
score_labels = []
for key, group in groups:
    group.sort_values(by="load_day", ascending=False, inplace=True)
    if key not in new_dict:
        try:
            new_dict[key] = (group["load_day"].dropna(how="any").iloc[0], group.shape[0])
        except:
            new_dict[key] = None

for index, row in df_score.iterrows():
    most_recent_date_arr.append(new_dict[row["id_driver"]][0])
    num_trips_arr.append(new_dict[row["id_driver"]][1])
    if row["id_driver"] in label_dict:
        score_labels.append(label_dict[row["id_driver"]])
    else:
        print("Not here: ", row["id_driver"])
        rand_num = random.randint(0, 1000)
        if rand_num <= 125:
            score_labels.append(1)
            label_dict[row["id_driver"]] = 1
        else:
            score_labels.append(0)
            label_dict[row["id_driver"]] = 0

score_labels = np.array(score_labels)
# df_score["most_recent_load_date"] = np.array(most_recent_date_arr)
# df_score["num_trips_made"] = np.array(num_trips_arr)

Not here:  1288
Not here:  7594
Not here:  7623
Not here:  8205
Not here:  10359
Not here:  11514
Not here:  12429
Not here:  13711
Not here:  13761
Not here:  15073
Not here:  16642
Not here:  20357
Not here:  26545
Not here:  29070
Not here:  29653
Not here:  31557
Not here:  31873
Not here:  35563
Not here:  35572


In [149]:
display(df_score["num_trips_made"])

0      3
1      3
2      1
3      1
4      1
      ..
663    1
664    1
665    1
666    1
667    1
Name: num_trips_made, Length: 668, dtype: Int64

In [150]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 668 non-null    Int64         
 1   dim_carrier_type          668 non-null    string        
 2   dim_carrier_company_name  668 non-null    string        
 3   carrier_trucks            668 non-null    string        
 4   signup_source             668 non-null    string        
 5   ts_signup                 668 non-null    string        
 6   ts_first_approved         572 non-null    string        
 7   days_signup_to_approval   572 non-null    Int64         
 8   home_base_city            668 non-null    string        
 9   home_base_state           668 non-null    string        
 10  num_trucks                668 non-null    float64       
 11  interested_in_drayage     668 non-null    string        
 12  port_qualified        

In [151]:
# # NO: dt, weekday, year, id_carrier_number, dim_preferred_lanes, load_day, loads



# ["id_driver", "dim_carrier_type", "dim_carrier_company_name", "home_base_city", "home_base_state", 
#                 "carrier_trucks", "signup_source", "ts_signup", "ts_first_approved",
#                 "days_signup_to_approval", "num_trucks", "interested_in_drayage", 
#                 "port_qualified", "driver_with_twic", 
#                 "first_load_date", "most_recent_load_date", "marketplace_loads_otr", 
#                 "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
#                 "brokerage_loads_atlas", "brokerage_loads", "total_loads", "num_trips_made", "label"]

drop_cols = ["dt", "weekday", "year", "id_carrier_number", "dim_preferred_lanes", "load_day", "loads"]
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [152]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id_driver                 668 non-null    Int64         
 1   dim_carrier_type          668 non-null    string        
 2   dim_carrier_company_name  668 non-null    string        
 3   carrier_trucks            668 non-null    string        
 4   signup_source             668 non-null    string        
 5   ts_signup                 668 non-null    string        
 6   ts_first_approved         572 non-null    string        
 7   days_signup_to_approval   572 non-null    Int64         
 8   home_base_city            668 non-null    string        
 9   home_base_state           668 non-null    string        
 10  num_trucks                668 non-null    float64       
 11  interested_in_drayage     668 non-null    string        
 12  port_qualified        

# 3. Basic Statistics

In [153]:
corr_matrix = df.corr()
for col_name in (list(df.columns.values)): # prints all the correlation matrices corresponding to each feature
    try:
        print(col_name)
        display(corr_matrix[col_name].sort_values(ascending=False))
        print('---------------------------------------------------------------------')
    except:
        print("{} is not of type integer".format(col_name))
        print('---------------------------------------------------------------------')

id_driver


id_driver                  1.000000
days_since_last_load       0.839446
num_trucks                 0.169567
brokerage_loads_atlas      0.145307
marketplace_loads_atlas    0.138116
marketplace_loads          0.123344
label                      0.075683
total_loads                0.052722
num_trips_made             0.018007
marketplace_loads_otr     -0.012441
brokerage_loads           -0.037352
brokerage_loads_otr       -0.063795
days_signup_to_approval   -0.734778
Name: id_driver, dtype: float64

---------------------------------------------------------------------
dim_carrier_type
dim_carrier_type is not of type integer
---------------------------------------------------------------------
dim_carrier_company_name
dim_carrier_company_name is not of type integer
---------------------------------------------------------------------
carrier_trucks
carrier_trucks is not of type integer
---------------------------------------------------------------------
signup_source
signup_source is not of type integer
---------------------------------------------------------------------
ts_signup
ts_signup is not of type integer
---------------------------------------------------------------------
ts_first_approved
ts_first_approved is not of type integer
---------------------------------------------------------------------
days_signup_to_approval


days_signup_to_approval    1.000000
brokerage_loads_otr        0.048116
brokerage_loads            0.027929
num_trips_made             0.010462
marketplace_loads_otr      0.007436
total_loads               -0.043765
num_trucks                -0.053498
label                     -0.064084
marketplace_loads         -0.089268
brokerage_loads_atlas     -0.092088
marketplace_loads_atlas   -0.099058
days_since_last_load      -0.628732
id_driver                 -0.734778
Name: days_signup_to_approval, dtype: float64

---------------------------------------------------------------------
home_base_city
home_base_city is not of type integer
---------------------------------------------------------------------
home_base_state
home_base_state is not of type integer
---------------------------------------------------------------------
num_trucks


...

---------------------------------------------------------------------
days_since_last_load


days_since_last_load       1.000000
id_driver                  0.839446
num_trips_made             0.284921
marketplace_loads          0.236472
total_loads                0.216076
marketplace_loads_atlas    0.202243
brokerage_loads_atlas      0.179735
num_trucks                 0.163449
marketplace_loads_otr      0.157085
label                      0.142288
brokerage_loads            0.092076
brokerage_loads_otr        0.061265
days_signup_to_approval   -0.628732
Name: days_since_last_load, dtype: float64

---------------------------------------------------------------------
label


label                      1.000000
marketplace_loads          0.584442
marketplace_loads_atlas    0.577596
total_loads                0.541811
num_trips_made             0.481593
brokerage_loads_atlas      0.327819
brokerage_loads            0.293505
brokerage_loads_otr        0.239032
marketplace_loads_otr      0.163324
days_since_last_load       0.142288
id_driver                  0.075683
num_trucks                -0.035292
days_signup_to_approval   -0.064084
Name: label, dtype: float64

---------------------------------------------------------------------


 Also year and TODO_FIND_COLUMN_NAME_2 and year are highly correlated and have a similar impact on label, so we could drop one? 

Is there really a need for brokerage_loads when it is so highly correlated to brokerage_loads_otr due to the vast majority of shipments being delivered over-the-road as compared to via ATLAS? 

I have the same question about total_loads due to the vast majority of loads being brokerage loads...

What's the point of having both year and date?

We can remove the id_carrier_number column from this dataset as it is not relevant to predicting a label of 0 or 1 (When trying to find high performing drivers, we need to know their carrier number, so we can extract the id_carrier_number column for now...)

We could one-hot-encode sign-up source and see its effect on labels.

We can remove the ts_first_approved column because the date of approval shouldn't matter that much but instead the days_signup_to_approval matter.

dim_preferred_lanes only has a few values so we can either remove the column or impute values.

Also first_load_date, most_recent_load_date and load_day shouldn't matter much. Instead we can have values such as: number of days doing the job = most_recent_load_date - first_load_date
AND
days_from_last_load_to_today = todays_date - most_recent_load_date

There are also a couple other features we need to impute.

Also, only people that are port qualified can provide drayage services, so we should create a field called qualified_and_interest_in_drayage which is only 1 (yes) when interested_in_drayage = "yes" and port_qualified = "yes". We can also cross these features...

# 4. Data Feature Extraction Plan and Pipeline

In [154]:
# df["location"] = list(zip(df["home_base_city"], df["home_base_state"]))# feature cross to get (city, state) tuple
# # feature cross for interested in drayage and port qualified
# df["drayage_interested_port_qualified"] = list(zip(df["interested_in_drayage"], df["port_qualified"]))
# display(df["location"])
# display(df["drayage_interested_port_qualified"])

In [155]:
def drayage_feature_cross(df):
    loc_cross = list(zip(df["home_base_city"], df["home_base_state"]))# feature cross to get (city, state) tuple
    # feature cross for interested in drayage and port qualified
    drayage_cross = list(zip(df["interested_in_drayage"], df["port_qualified"]))

    drayage_arr = []
    for list_item in drayage_cross:
        if list_item[0] == "yes" and list_item[1] == "yes":
            drayage_arr.append("000001")
        if list_item[0] == "yes" and list_item[1] == "no":
            drayage_arr.append("000010")
        if list_item[0] == "no" and list_item[1] == "yes":
            drayage_arr.append("000100")
        if list_item[0] == "no" and list_item[1] == "no":
            drayage_arr.append("001000")
        if list_item[0] == "not specified" and list_item[1] == "yes":
            drayage_arr.append("010000")
        if list_item[0] == "not specified" and list_item[1] == "no":
            drayage_arr.append("100000")

    df["drayage_interested_port_qualified"] = np.array(drayage_arr)
    display(df["drayage_interested_port_qualified"])

drayage_feature_cross(df)
drayage_feature_cross(df_score)

0       100000
1       100000
2       100000
3       100000
4       100000
         ...  
5286    100000
5287    000001
5288    000001
5289    000001
5290    000001
Name: drayage_interested_port_qualified, Length: 5291, dtype: object

0      100000
1      100000
2      100000
3      100000
4      100000
        ...  
663    100000
664    000001
665    000010
666    000010
667    010000
Name: drayage_interested_port_qualified, Length: 668, dtype: object

In [156]:
id_driver_number_col = np.array(df["id_driver"]) # extract id_driver column
id_driver_number_col_score = np.array(df_score["id_driver"]) # extract id_driver column

drop_cols = ["id_driver", "interested_in_drayage", "port_qualified", 
             "ts_signup", "ts_first_approved"] #"home_base_city","home_base_state",
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   dim_carrier_type                   5291 non-null   string        
 1   dim_carrier_company_name           5284 non-null   string        
 2   carrier_trucks                     5291 non-null   string        
 3   signup_source                      5291 non-null   string        
 4   days_signup_to_approval            3962 non-null   Int64         
 5   home_base_city                     5291 non-null   string        
 6   home_base_state                    5291 non-null   string        
 7   num_trucks                         5249 non-null   float64       
 8   driver_with_twic                   5291 non-null   string        
 9   first_load_date                    5291 non-null   datetime64[ns]
 10  most_recent_load_date              5

In [158]:
names = {}
for index, row in df.iterrows():
    names[row["dim_carrier_company_name"]] = int(names.get(row["dim_carrier_company_name"], 0) + 1)
listo = list(names.items())
listo.sort(reverse=True, key=lambda x: int(x[1]))
#listo[:50]

In [159]:
listo = listo[:50]
count_50 = sum([x[1] for x in listo])
print("# 50: ", count_50)
print("Percentage 50: ", count_50/len(names))

names_arr = [tuples[0] for tuples in listo]
print(names_arr)

# 50:  1290
Percentage 50:  0.5176565008025682
['NFS asset Drayage', 'MC Express Trucking LLC', 'Roadrunner Transportation', 'Dong Fang Marketing Inc', 'ROADMOND LOGISTICS INC.', 'Consistent Trucking Inc', 'BLUE FREIGHT TRANSPORT INC', 'Convoy Express', 'Mega Fleet', 'iDC Drayage', 'Cross World Logistics', 'USA Diamonds Trucking', 'pointdirect', 'Chaidez Trucking', 'Saia LTL Freight', 'American Better Choice Corporation', 'American Freightways Lp.', 'Fastrucking', 'MERIDIAN LOGISTICS INC', 'Carlos Flores', 'J&G Transportation Group Inc', 'KLF transport inc', 'Star Rain LLC', 'Starco Logistics Inc.', '664 Transport', 'FTS EXPRESS INC', 'IDC OTR', "Luna's Transportation group", 'AGRAMONT TRANSPORT INC', 'BGood VirtueT Inc', 'Geber Freight', 'R&Y Castellanos Trucking Inc.', 'JC Transport', 'Great Qin Transportation LLC', 'JM Express Inc', 'MT Brothers Groups', 'cbt trucking', 'Road Eagle Logistics Corp', 'O.A. EXPRESS INC', 'AMPAK Logistics INC.', 'Kuang Trucking Inc.', 'nolan transportat

In [160]:
def bucketize(df):
    days_worked = []
    for index, row in df.iterrows(): # bucketize the most frequent dim_carrier_company names, 
                                     # put less frequent names in a single bucket
        try:
            if row["dim_carrier_company_name"] not in names_arr:
                df.at[index, "dim_carrier_company_name"] = "Other"
        except:
            df.at[index, "dim_carrier_company_name"] = "Other"

        # find number of days driver has worked
        if row["most_recent_load_date"] != np.nan and row["first_load_date"] != np.nan:
            days_worked.append((row["most_recent_load_date"] - row["first_load_date"]).days)
        else:
            days_worked.append(None)
    df["days_tenured"] = np.array(days_worked)

bucketize(df)
bucketize(df_score)

In [161]:
# # Get total_loads for df_score in order to find labels
# total_loads = []
# for index, row in df_score.iterrows():
#     total_loads.append(row["marketplace_loads"] + row["brokerage_loads"])
# df_score["total_loads"] = np.array(total_loads)

In [162]:
# total_loads_score_75 = df_score.total_loads.quantile(0.75) # finds 75th percentile of loads
# most_recent_load_date_score_75 = df_score.most_recent_load_date.quantile(0.75) # finds 75th percentile of most recent load date

# print(total_loads_score_75)
# print(most_recent_load_date_score_75)

In [163]:
# score_labels = []
# for index, row in df_score.iterrows(): # changes the labels in the label columns
#     # checks if the load and most recent load date are in the 75th percentile
#     if row["total_loads"] >= total_loads_score_75 and row["most_recent_load_date"] >= most_recent_load_date_score_75:
#         score_labels.append(1)
#     else:
#         score_labels.append(0)
# print(len(score_labels))
# print(score_labels)

In [164]:
drop_cols = ["most_recent_load_date", "first_load_date", "weekday", "load_day", "total_loads"]
for col in drop_cols:
    try:
        df_score = df_score.drop(columns=[col])
    except:
        continue

for col in drop_cols:
    try:
        df = df.drop(columns=[col])
    except:
        continue

In [165]:
df_unlabeled = df.drop(columns=["label"])
labels = df["label"].copy()

In [166]:
uniqueValues, occurCount = np.unique(labels, return_counts=True)
print("Unique Values : " , uniqueValues)
print("Occurrence Count : ", occurCount)

Unique Values :  [0 1]
Occurrence Count :  [5236   55]


In [167]:
df_unlabeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5291 entries, 0 to 5290
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dim_carrier_type                   5291 non-null   string 
 1   dim_carrier_company_name           5291 non-null   string 
 2   carrier_trucks                     5291 non-null   string 
 3   signup_source                      5291 non-null   string 
 4   days_signup_to_approval            3962 non-null   Int64  
 5   home_base_city                     5291 non-null   string 
 6   home_base_state                    5291 non-null   string 
 7   num_trucks                         5249 non-null   float64
 8   driver_with_twic                   5291 non-null   string 
 9   marketplace_loads_otr              5291 non-null   Int64  
 10  marketplace_loads_atlas            5291 non-null   Int64  
 11  marketplace_loads                  5291 non-null   Int64

In [168]:
df_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dim_carrier_type                   668 non-null    string 
 1   dim_carrier_company_name           668 non-null    string 
 2   carrier_trucks                     668 non-null    string 
 3   signup_source                      668 non-null    string 
 4   days_signup_to_approval            572 non-null    Int64  
 5   home_base_city                     668 non-null    string 
 6   home_base_state                    668 non-null    string 
 7   num_trucks                         668 non-null    float64
 8   driver_with_twic                   668 non-null    string 
 9   marketplace_loads_otr              668 non-null    Int64  
 10  marketplace_loads_atlas            668 non-null    Int64  
 11  marketplace_loads                  668 non-null    Int64  

In [169]:
cols = list(df_unlabeled.columns.values)
print(cols)

['dim_carrier_type', 'dim_carrier_company_name', 'carrier_trucks', 'signup_source', 'days_signup_to_approval', 'home_base_city', 'home_base_state', 'num_trucks', 'driver_with_twic', 'marketplace_loads_otr', 'marketplace_loads_atlas', 'marketplace_loads', 'brokerage_loads_otr', 'brokerage_loads_atlas', 'brokerage_loads', 'num_trips_made', 'days_since_last_load', 'drayage_interested_port_qualified', 'days_tenured']


In [170]:
df_score = df_score[cols]

df_score['num_trucks'] = pd.to_numeric(df_score['num_trucks'], errors='coerce')
df_score['days_signup_to_approval'] = pd.to_numeric(df_score['days_signup_to_approval'], errors='coerce')

df_unlabeled['num_trucks'] = pd.to_numeric(df_unlabeled['num_trucks'], errors='coerce')
df_unlabeled['days_signup_to_approval'] = pd.to_numeric(df_unlabeled['days_signup_to_approval'], errors='coerce')

# df_unlabeled['home_base_city'].astype(str) #= to_string(df_unlabeled['home_base_city'], errors='coerce')
# df_unlabeled['home_base_state'].astype(str) #= pd.to_string(df_unlabeled['home_base_state'], errors='coerce')

convert = ["id_driver", "days_signup_to_approval", "marketplace_loads_otr", 
               "marketplace_loads_atlas", "marketplace_loads", "brokerage_loads_otr",
               "brokerage_loads_atlas", "brokerage_loads", "num_trips_made",
               "num_trucks", "dim_carrier_type", "dim_carrier_company_name",
               "interested_in_drayage", "port_qualified", "signup_source", "driver_with_twic",
          "home_base_city","home_base_state"] #"home_base_city","home_base_state"
for col in convert:
    try:
        df_score[col] = df_score[col].convert_dtypes()
    except:
        continue

for col in convert:
    try:
        df_unlabeled[col] = df_unlabeled[col].convert_dtypes()
    except:
        continue

In [171]:
df_concat = pd.concat([df_unlabeled, df_score], ignore_index=True)
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5959 entries, 0 to 5958
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dim_carrier_type                   5959 non-null   string 
 1   dim_carrier_company_name           5959 non-null   string 
 2   carrier_trucks                     5959 non-null   string 
 3   signup_source                      5959 non-null   string 
 4   days_signup_to_approval            4534 non-null   Int64  
 5   home_base_city                     5959 non-null   string 
 6   home_base_state                    5959 non-null   string 
 7   num_trucks                         5917 non-null   float64
 8   driver_with_twic                   5959 non-null   string 
 9   marketplace_loads_otr              5959 non-null   Int64  
 10  marketplace_loads_atlas            5959 non-null   Int64  
 11  marketplace_loads                  5959 non-null   Int64

In [172]:
for col in convert:
    try:
        df_concat[col] = df_concat[col].convert_dtypes()
    except:
        continue

In [173]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5959 entries, 0 to 5958
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   dim_carrier_type                   5959 non-null   string 
 1   dim_carrier_company_name           5959 non-null   string 
 2   carrier_trucks                     5959 non-null   string 
 3   signup_source                      5959 non-null   string 
 4   days_signup_to_approval            4534 non-null   Int64  
 5   home_base_city                     5959 non-null   string 
 6   home_base_state                    5959 non-null   string 
 7   num_trucks                         5917 non-null   float64
 8   driver_with_twic                   5959 non-null   string 
 9   marketplace_loads_otr              5959 non-null   Int64  
 10  marketplace_loads_atlas            5959 non-null   Int64  
 11  marketplace_loads                  5959 non-null   Int64

In [174]:
imputer = IterativeImputer()
# df = imputer.fit_transform(df)
categorical_features_one_hot = ["dim_carrier_type", "dim_carrier_company_name", "carrier_trucks", 
                                "signup_source", "driver_with_twic", "home_base_city","home_base_state"] #"home_base_city","home_base_state"

df_num = df_concat.drop(columns=categorical_features_one_hot)
# df_num = df_concat
numerical_features = list(df_num)

num_pipeline = Pipeline([
        ('imputer', imputer),
        ('std_scaler', StandardScaler()),
    ])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, numerical_features),
        ("cat", OneHotEncoder(sparse=False), categorical_features_one_hot), #sparse=False
    ])
df_prepared = full_pipeline.fit_transform(df_concat)

In [175]:
display(df_prepared)

array([[ 1.89676035, -0.39804762, -0.23261086, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.90882444, -0.39804762, -0.23261086, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.92261463, -0.39804762, -0.23261086, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.01120964, -0.39804762, -0.23261086, ...,  0.        ,
         0.        ,  0.        ],
       [-1.01120964, -0.37456332, -0.15643114, ...,  0.        ,
         0.        ,  0.        ],
       [-1.01120964, -0.39804762, -0.23261086, ...,  0.        ,
         0.        ,  0.        ]])

In [176]:
# X_train_temp = df_prepared[:5291]
# y_train_temp = labels
# X_test = df_prepared[5291:]
# y_test = score_labels

# y_train_temp = y_train_temp.astype('int')

In [208]:
X = df_prepared[:5291]
y = labels
X_test_score = df_prepared[5291:]
y_test_score = score_labels

y = y.astype('int')
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [209]:
X_train, y_train = SMOTE().fit_resample(X_train_temp, y_train_temp)

In [210]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(8374, 647) (1059, 647)
(8374,) (1059,)


# 5. Linear Regression

In [211]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lr_predicted = lin_reg.predict(X_test)

In [212]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, lr_predicted)
rmse = np.sqrt(mse)
rmse

294667777.6756903

In [213]:
from sklearn.metrics import r2_score

r2_score(y_test, lr_predicted)

-9.282858436633491e+18

In [214]:
import statsmodels.api as sm

X_new = sm.add_constant(X_train)
toyregr_sm = sm.OLS(y_train.astype(float), X_new.astype(float))
results_sm = toyregr_sm.fit()

print(results_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                  label   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                     147.9
Date:                Thu, 18 Mar 2021   Prob (F-statistic):               0.00
Time:                        02:50:28   Log-Likelihood:                 4037.3
No. Observations:                8374   AIC:                            -6993.
Df Residuals:                    7833   BIC:                            -3188.
Df Model:                         540                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0403      0.009      4.500      0.0

In [215]:
from sklearn.linear_model import LogisticRegression

y_temp = y_train.astype('int')

log_reg = LogisticRegression(max_iter=1000, multi_class='ovr')
log_reg.fit(X_train, y_temp)
log_predicted = log_reg.predict(X_test)

In [216]:
mse = mean_squared_error(y_test, log_predicted)
rmse = np.sqrt(mse)
rmse

0.1064492590824699

In [217]:
r2_score(y_test, log_predicted)

-0.21143946615824594

In [218]:
accuracy = (accuracy_score(y_test, log_predicted))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, log_predicted)))
print("F1-score = {}".format(f1_score(y_test, log_predicted)))

accuracy = 0.9886685552407932%
roc_auc_score = 0.994280266920877
F1-score = 0.625


# 6. PCA

In [219]:
pca = PCA(n_components=0.95) # Create an instance of PCA model
pca.fit(X_train) # Fit X_train to PCA
X_train = pca.transform(X_train) # transform training data
X_test = pca.transform(X_test) # transform test data
print(pca.explained_variance_)
print(pca.n_components_)

[35.52613598 30.52934619  6.7222188   3.8818686   1.3606588   1.32032989]
6


In [220]:
print(X_train)

[[-3.87320299 -1.63360314 -1.02941636  0.83444497 -0.36030468  1.71501823]
 [-4.1430277  -2.4140799   0.04275821 -0.78489977  1.36622729 -1.24442034]
 [-4.24327697 -2.0921594  -0.67651334 -0.12336708  0.26108134  1.24673239]
 ...
 [19.13086715 -0.23106214 -5.40819249 -1.48894115 -0.42373005  0.88548112]
 [ 6.34595162 -0.05757893  9.96182115  1.72733095 -0.45550872  1.20243083]
 [-2.73678844  7.26685418 -0.82424373  1.77636817 -1.11934945 -1.35478083]]


In [221]:
print(X_test)

[[-4.92228915 -2.4908769  -0.54466133 -1.14976827 -1.51749068  2.08461436]
 [-4.63600142 -2.40993292 -0.52573366 -0.73827486 -0.84772528  1.57207674]
 [-4.9407654  -2.50070167 -0.52851711 -1.19354067 -1.15177872  1.6575278 ]
 ...
 [-4.51381109 -2.46410533 -0.2326742  -1.00715291  0.45151603  0.09787329]
 [-4.59921061 -2.4795135  -0.27687645 -1.11036168 -0.02900953  0.43898091]
 [-4.91255806 -2.49893103 -0.52150667 -1.16306334 -1.28959169  1.86178245]]


In [222]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(8374, 6) (1059, 6)
(8374,) (1059,)


# 7. Ensemble

In [ ]:
# y_train = y_train.astype('int')
# y_test = y_test.astype('int')

In [235]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score

# assuming we have X_train,X_test,y_train,y_test at this time
# I first run Random Forest using random hard coded settings to get a baseline
rf = RandomForestClassifier(n_estimators=80,max_depth=7,max_features=3)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
test_score = r2_score(y_test,y_pred)
print(test_score)
accuracy = (accuracy_score(y_test, y_pred))
print("accuracy = {}%".format(accuracy))
print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
print("F1-score = {}".format(f1_score(y_test, y_pred)))

-0.6152526215443281
accuracy = 0.9848914069877243%
roc_auc_score = 0.9428503336510962
F1-score = 0.5294117647058825


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# I then use RandomizedSearchCV to find the optimal hyperparameters
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1500, num = 30)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(5, 100, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 7, 10]
min_samples_leaf = [2, 5, 7, 10]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf = RandomForestClassifier(class_weight='balanced', random_state=42)

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
rf = RandomForestRegressor(n_estimators=411,max_depth=68,max_features='auto',min_samples_split=5,min_samples_leaf=2,bootstrap=True)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
misclassified = 0
for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
        misclassified += 1
accuracy = ((1000-misclassified)*100/len(y_pred))
print("accuracy = {}%".format(accuracy))

In [ ]:
# from sklearn.datasets import make_regression
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score

# # assuming we have X_train,X_test,y_train,y_test at this time
# # I first run Random Forest using random hard coded settings to get a baseline
# rf = RandomForestRegressor(n_estimators=80,max_depth=7,max_features=3)
# rf.fit(X_train,y_train)
# y_pred = rf.predict(X_test)
# test_score = r2_score(y_test,y_pred)
# test_score

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# import numpy as np
# # I then use RandomizedSearchCV to find the optimal hyperparameters
# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1500, num = 10)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(5, 100, num = 10)]
# max_depth.append(None)
# min_samples_split = [2, 5, 7, 10, 20]
# min_samples_leaf = [2, 5, 7, 10]
# bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(X_train, y_train)
# rf_random.best_params_

In [ ]:
# # I then output the r2 score again as a sanity check to verify that my RanomdizedSearchCV actually did find the best settings
# rf = RandomForestRegressor(n_estimators=411,max_depth=68,max_features='auto',min_samples_split=5,min_samples_leaf=2,bootstrap=True)
# rf.fit(X_train,y_train)
# y_pred = rf.predict(X_test)
# from sklearn.metrics import r2_score
# test_score = r2_score(y_test,y_pred)
# test_score

In [ ]:
# we then use the hyperparameters we found from the RandomizedSearchCV to do a second more thorough check around that range
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [65, 68, 71],
    'max_features': ['auto'],
    'min_samples_leaf': [2, 4, 6],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [375, 411, 500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [ ]:
# we then output the results using the optimal hyperparameters to check that our model has improved
rf = RandomForestRegressor(n_estimators=375,max_depth=68,max_features='auto',min_samples_split=3,min_samples_leaf=2,bootstrap=True)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
from sklearn.metrics import r2_score
test_score = r2_score(y_test,y_pred)
test_score

In [ ]:
# AdaBoost using the same settings
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

dt = DecisionTreeRegressor(max_depth=80,max_features='auto',min_samples_leaf=2,min_samples_split=3)
ab = AdaBoostRegressor(base_estimator=dt,learning_rate=1,n_estimators=450)
ab.fit(X_train,y_train)
y_pred = ab.predict(X_test)
test_score = r2_score(y_test,y_pred)
test_score

# 8. Neural Network Classifier

In [ ]:
# # Define a transform to normalize the data
# transform = transforms.Compose([transforms.ToTensor(),
#                                 transforms.Normalize((0.5,), (0.5,)),
#                               ])
# Download and load the training data
NN_X_train = torch.tensor(df.drop(["label"], axis=1).values)
NN_y_train = torch.tensor(df["label"].values)
trainset = TensorDataset(NN_X_train, NN_y_train)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
NUM_FEATURES = 20
NUM_HIDDEN1_NODES = 400
NUM_EPOCHS = 30

model = nn.Sequential(nn.Linear(NUM_FEATURES, NUM_HIDDEN1_NODES),
                      nn.Sigmoid(),
                      nn.Linear(NUM_HIDDEN1_NODES, 1)
                     )

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

for e in range(NUM_EPOCHS):
    running_loss = 0
    for data, labels in trainloader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Training loss: {running_loss/len(trainloader)}")
    
# class NeuralNet(nn.Module):
#     def __init__(self):
#         super(NeuralNet, self).__init__()
        
#         # Inputs to hidden layer linear transformation
#         self.hidden1 = nn.Linear(NUM_FEATURES, NUM_HIDDEN1_NODES)
#         self.output = nn.Linear(NUM_HIDDEN1_NODES, 1)
        
#         self.sigmoid = nn.Sigmoid()
        
#     def forward(self, x):
#         x = self.hidden1(x)
#         x = self.sigmoid(x)
#         x = self.output(x)
#         x = self.softmax(x)
#         return x

In [ ]:
# model = Network()
# model

# 9. Cross-Validate

In [ ]:
from sklearn import model_selection

kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)

sample_model_kfold = sample()

sample_results_kfold = model_selection.cross_val_score(sample_model_kfold, df_prepared, df_labels, cv=kfold)

# 10. Custom Model

In [233]:
kernel_types = ['linear', 'rbf', 'sigmoid']
for kernel in kernel_types:
    svc = SVC(kernel=kernel, class_weight='balanced', random_state=42, max_iter=100) #class_weight='balanced'
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    accuracy = (accuracy_score(y_test, y_pred))
    print("accuracy for kernel={} = {}%".format(kernel, accuracy))
    print("roc_auc_score = {}".format(roc_auc_score(y_test, y_pred)))
    print("F1-score = {}".format(f1_score(y_test, y_pred)))

accuracy for kernel=linear = 0.947119924457035%
roc_auc_score = 0.8247378455672069
F1-score = 0.2
accuracy for kernel=rbf = 0.9886685552407932%
roc_auc_score = 0.994280266920877
F1-score = 0.625
accuracy for kernel=sigmoid = 0.9612842304060434%
roc_auc_score = 0.9804575786463298
F1-score = 0.3278688524590164


/Users/shrenik/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
/Users/shrenik/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
/Users/shrenik/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
